<img src="https://i.ibb.co/2dkgvqY/visualizing.png">
<img src="https://i.ibb.co/vw8Fvgz/Maya-Ayoubi-Font.png">
<center><a href="http://bit.ly/vizvax2021">bit.ly/vizvax2021</a></center>
<img src="https://www.gannett-cdn.com/presto/2020/03/07/USAT/97bc2b03-be50-4814-9389-9b5e52e3045b-promo-art.png?crop=5658,3183,x794,y621&width=1200" width="400" height="50" >

In [1]:
import pandas as pd
import plotly.express as px
import getpass
import os
import fbprophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

In [2]:
ACCESS_CODE = getpass.getpass('kaggle username: ')
SECRET_CODE = getpass.getpass('kaggle API token: ')

kaggle username: ········
kaggle API token: ········


In [121]:
os.environ['KAGGLE_USERNAME'] = ACCESS_CODE
os.environ['KAGGLE_KEY'] = SECRET_CODE

import kaggle as kg

In [122]:
kg.api.authenticate()
kg.api.dataset_download_files(dataset='gpreda/covid-world-vaccination-progress', path='gt.zip', unzip=True)
country_df = pd.read_csv('gt.zip/country_vaccinations.csv', encoding='ISO-8859-1')

kg.api.dataset_download_files(dataset='erikbruin/countries-of-the-world-iso-codes-and-population', path='gt.zip', unzip=True)
country_pop_df = pd.read_csv('gt.zip/countries_by_population_2019.csv', encoding='ISO-8859-1')

kg.api.dataset_download_files(dataset='erikbruin/countries-of-the-world-iso-codes-and-population', path='gt.zip', unzip=True)
iso_df = pd.read_csv('gt.zip/country_codes_2020.csv', encoding='ISO-8859-1')

In [123]:
states_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv'
us_states_df = pd.read_csv(states_url)

us_vaccinations_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/country_data/United%20States.csv'
us_vaccinations_df = pd.read_csv(us_vaccinations_url)

state_postal_url = 'https://raw.githubusercontent.com/ayoubimaya/COVID19-Vaccination-Progress/main/datasets/State_Postal_Codes.csv'
state_postal_df = pd.read_csv(state_postal_url)

<img src="https://i.ibb.co/fnprQrX/exploration.png">

In [124]:
country_df.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.00,0.00,nan,nan,nan,0.00,0.00,nan,nan,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,nan,nan,nan,nan,1367.00,nan,nan,nan,35.00,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,nan,nan,nan,nan,1367.00,nan,nan,nan,35.00,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,nan,nan,nan,nan,1367.00,nan,nan,nan,35.00,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,nan,nan,nan,nan,1367.00,nan,nan,nan,35.00,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/


In [125]:
country_df.shape

(18764, 15)

In [126]:
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18764 entries, 0 to 18763
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              18764 non-null  object 
 1   iso_code                             18764 non-null  object 
 2   date                                 18764 non-null  object 
 3   total_vaccinations                   10795 non-null  float64
 4   people_vaccinated                    10038 non-null  float64
 5   people_fully_vaccinated              7667 non-null   float64
 6   daily_vaccinations_raw               8998 non-null   float64
 7   daily_vaccinations                   18551 non-null  float64
 8   total_vaccinations_per_hundred       10795 non-null  float64
 9   people_vaccinated_per_hundred        10038 non-null  float64
 10  people_fully_vaccinated_per_hundred  7667 non-null   float64
 11  daily_vaccinations_per_milli

In [127]:
country_df.isna().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                      7969
people_vaccinated                       8726
people_fully_vaccinated                11097
daily_vaccinations_raw                  9766
daily_vaccinations                       213
total_vaccinations_per_hundred          7969
people_vaccinated_per_hundred           8726
people_fully_vaccinated_per_hundred    11097
daily_vaccinations_per_million           213
vaccines                                   0
source_name                                0
source_website                             0
dtype: int64

In [128]:
us_states_df.shape

(8343, 14)

In [129]:
us_states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8343 entries, 0 to 8342
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 8343 non-null   object 
 1   location                             8343 non-null   object 
 2   total_vaccinations                   8073 non-null   float64
 3   total_distributed                    7939 non-null   float64
 4   people_vaccinated                    7927 non-null   float64
 5   people_fully_vaccinated_per_hundred  7443 non-null   float64
 6   total_vaccinations_per_hundred       7545 non-null   float64
 7   people_fully_vaccinated              7845 non-null   float64
 8   people_vaccinated_per_hundred        7520 non-null   float64
 9   distributed_per_hundred              7532 non-null   float64
 10  daily_vaccinations_raw               8278 non-null   float64
 11  daily_vaccinations            

In [130]:
us_states_df.isna().sum()

date                                     0
location                                 0
total_vaccinations                     270
total_distributed                      404
people_vaccinated                      416
people_fully_vaccinated_per_hundred    900
total_vaccinations_per_hundred         798
people_fully_vaccinated                498
people_vaccinated_per_hundred          823
distributed_per_hundred                811
daily_vaccinations_raw                  65
daily_vaccinations                      65
daily_vaccinations_per_million         608
share_doses_used                       404
dtype: int64

In [131]:
us_vaccinations_df.shape

(136, 7)

In [132]:
us_vaccinations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   location                 136 non-null    object 
 1   date                     136 non-null    object 
 2   vaccine                  136 non-null    object 
 3   source_url               136 non-null    object 
 4   total_vaccinations       136 non-null    int64  
 5   people_vaccinated        135 non-null    float64
 6   people_fully_vaccinated  121 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 7.6+ KB


In [133]:
us_vaccinations_df.isna().sum()

location                    0
date                        0
vaccine                     0
source_url                  0
total_vaccinations          0
people_vaccinated           1
people_fully_vaccinated    15
dtype: int64

In [134]:
state_postal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   state        51 non-null     object
 1   postal_code  51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [135]:
country_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rank        232 non-null    int64  
 1   name        232 non-null    object 
 2   pop2019     232 non-null    float64
 3   pop2018     0 non-null      float64
 4   GrowthRate  232 non-null    float64
 5   area        232 non-null    float64
 6   Density     232 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 12.8+ KB


In [136]:
iso_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    232 non-null    object
 1   cca2    231 non-null    object
 2   cca3    232 non-null    object
 3   ccn3    232 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.4+ KB


In [137]:
country_limit = len(country_df) * .50
country_df = country_df.dropna(thresh=country_limit, axis=1)

#us_limit = len(country_df) * .50
#us_states_df = us_states_df.dropna(thresh=us_limit, axis=1)

In [138]:
nans = country_df[country_df['iso_code'].isnull()]
nans['country'].unique()

array([], dtype=object)

In [139]:
country_df['iso_code'].fillna('GBR', inplace = True)

In [140]:
'''every combo of vaccine'''

country_df['vaccines'].unique()

array(['Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V',
       'Sputnik V', 'Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V',
       'Oxford/AstraZeneca, Sinovac, Sputnik V', 'Pfizer/BioNTech',
       'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Sinovac',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'Moderna, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac',
       'Oxford/AstraZeneca, Sinopharm/Beijing',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Covaxin, Oxford/AstraZeneca',
       'Sinopharm/Beijing, Sinopharm/Wuhan, Sinovac',
       'Covaxin, Oxford/AstraZeneca, Sinopharm/Beijing',
       'Moderna, Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V',
 

<img src="https://i.ibb.co/GWRgJcd/analysis.png">

<img src="https://i.ibb.co/0ZByV5r/mrna-title.png">
<img src="https://upload.wikimedia.org/wikipedia/commons/f/fe/Vaccine_candidate_mechanisms_for_SARS-CoV-2_%2849948301838%29.jpg" width="400" height="50" >

In [141]:
choropleth_grouping = country_df.groupby('country').max()

In [142]:
mrna_names = ['Moderna', 'Pfizer/BioNTech']
nonmrna_names = ['Oxford/AstraZeneca', 'Sinopharm/Beijing', 'Sinovac', 
                 'Sputnik V', 'Sinopharm/Wuhan', 'Covaxin', 'Johnson&Johnson', 'EpiVacCorona']
def mRNA(vaccine):
    has_mrna = False
    has_nonmrna = False
    
    for name in mrna_names:
        if name in vaccine:
            has_mrna = True
    for name in nonmrna_names:
        if name in vaccine:
            has_nonmrna = True
    
    if has_mrna and has_nonmrna:
        return 'Both'
    elif has_mrna:
        return 'mRNA'
    else:
        return 'Non-mRNA'
    
choropleth_grouping['type_of_vaccine'] = choropleth_grouping['vaccines'].apply(mRNA)

In [143]:
vacc_type_choro = px.choropleth(choropleth_grouping, 
                                locations='iso_code',
                                color='type_of_vaccine', 
                                color_discrete_sequence=px.colors.qualitative.G10, 
                                hover_name='type_of_vaccine')

vacc_type_choro.update_layout(title_text='Types of Vaccines Used', title_x=0.5)
vacc_type_choro.show()

In [144]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

vacc_type_choro.write_html('/home/mayoubi/COVID_data/Images/vacc_type_choro_html.html')

### To preview plot: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/vacc_type_choro_html.html

<img src="https://i.ibb.co/wCBWRfG/fastest-vaccines.png">

In [145]:
iso_pop_df = country_pop_df.merge(iso_df, left_on='name', right_on='name')
iso_pop_df = iso_pop_df[['name', 'cca3', 'pop2019']]
iso_pop_df['pop2019'] = iso_pop_df['pop2019'] * 1000
pd.options.display.float_format = "{:.2f}".format
iso_pop_df.head()

,name,cca3,pop2019
0,China,CHN,1433783686.00
1,India,IND,1366417754.00
2,United States,USA,329064917.00
3,Indonesia,IDN,270625568.00
4,Pakistan,PAK,216565318.00


In [146]:
country_df = country_df.merge(iso_pop_df, left_on='iso_code', right_on='cca3')

In [147]:
total_vacs_avg_df = country_df.groupby('iso_code').max()
vacs_per100_df = total_vacs_avg_df.sort_values(['total_vaccinations_per_hundred'], ascending=False)

'''filter for countries with populations greater than 10,000,0000'''
vacs_per100_df = vacs_per100_df[vacs_per100_df.pop2019 > 10_000_000].sort_values(['total_vaccinations_per_hundred'], ascending=False)
top_10_countries = vacs_per100_df[:10]

In [148]:
country_total_count = px.bar(top_10_countries, 
                             x=top_10_countries.index, 
                             y='total_vaccinations_per_hundred',
                             color='total_vaccinations_per_hundred', 
                             color_continuous_scale=px.colors.sequential.Emrld,
                             labels={
                                 'total_vaccinations_per_hundred': 'Total vaccinations (per hundred)',
                                 'location': ''
                             },)
country_total_count.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
country_total_count.update_coloraxes(showscale=False)
country_total_count.update_layout(title_text='Top 10 Countries (vaccinations per hundred)', title_x=0.5)
country_total_count.show()

In [149]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

country_total_count.write_html('/home/mayoubi/COVID_data/Images/country_total_count_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/country_total_count_html.html

In [150]:
daily_vacs_per_mil_df = country_df[['iso_code', 'country' ,'date', 'daily_vaccinations_per_million', 'pop2019']]
top_10_vacs = daily_vacs_per_mil_df.groupby('iso_code').max().sort_values(['daily_vaccinations_per_million'],ascending=False)

'''filter for countries with populations greater than 10,000,0000'''
top_10_vacs = top_10_vacs[top_10_vacs.pop2019 > 10_000_000].sort_values(['daily_vaccinations_per_million']
                                                                        ,ascending=False)[:10]
top_10_vacs

,country,date,daily_vaccinations_per_million,pop2019
iso_code,,,,
CHL,Chile,2021-05-17,15613.00,18952038.00
USA,United States,2021-05-18,10120.00,329064917.00
DOM,Dominican Republic,2021-05-16,10081.00,10738958.00
CAN,Canada,2021-05-18,9643.00,37411047.00
DEU,Germany,2021-05-18,9555.00,83517045.00
CHN,China,2021-05-18,9230.00,1433783686.00
GRC,Greece,2021-05-18,9144.00,10473455.00
BEL,Belgium,2021-05-18,9018.00,11539328.00
GBR,United Kingdom,2021-05-17,8872.00,67530172.00


In [151]:
top_10_countries = list(top_10_vacs.index)
countries_per_capita_df = daily_vacs_per_mil_df[daily_vacs_per_mil_df['iso_code'].isin(top_10_countries)]

In [152]:
daily_vacs_time_df = countries_per_capita_df.pivot_table(index='date', columns='iso_code', 
                                                       values='daily_vaccinations_per_million')
daily_vacs_time_df.reset_index(inplace=True)

In [153]:
daily_vacs_fig = px.line(daily_vacs_time_df, 
                         x='date', 
                         y=top_10_countries, 
                         color_discrete_sequence=px.colors.diverging.BrBG,
                         labels={
                             'date': ' ',
                             'value': ' '
                         })
daily_vacs_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
daily_vacs_fig.update_layout(title_text='Daily Vaccinations (per million)', title_x=0.5)
daily_vacs_fig.show()

In [154]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

daily_vacs_fig.write_html('/home/mayoubi/COVID_data/Images/daily_vacs_fig_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/daily_vacs_fig_html.html

<img src="https://i.ibb.co/3cNGKVf/states.png">

In [155]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming']

In [156]:
us_states_df = us_states_df[us_states_df['location'].isin(states)]

In [157]:
us_states_df['location'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming'], dtype=object)

In [158]:
us_states_choro = us_states_df.groupby('location').max()

In [159]:
us_states_choro = us_states_choro.reset_index()

In [160]:
def trim_all_columns(df):
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

state_postal_df = trim_all_columns(state_postal_df)

In [161]:
us_states_choro = us_states_choro.merge(state_postal_df, left_on='location', right_on='state')

In [162]:
states_choro = px.choropleth(us_states_choro, 
                             locations='postal_code', 
                             color='total_vaccinations_per_hundred',
                             hover_name='total_vaccinations_per_hundred', 
                             color_continuous_scale='Emrld',
                             locationmode='USA-states', scope='usa')

states_choro.update_layout(title_text='States Total Vaccination (per hundred)', title_x=0.5)
states_choro.show()

In [163]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

states_choro.write_html('/home/mayoubi/COVID_data/Images/states_choro_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/states_choro_html.html

In [164]:
us_total_fig = px.line(us_vaccinations_df, 
                       x='date', 
                       y='total_vaccinations',
                       labels={
                           'total_vaccinations': '# of total vaccinations',
                            'date': ' '
                       })
us_total_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
us_total_fig.update_layout(showlegend=False)
us_total_fig.update_layout(title_text='US Total Vaccinations', title_x=0.5)
us_total_fig.show()

In [165]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

us_total_fig.write_html('/home/mayoubi/COVID_data/Images/us_total_fig_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/us_total_fig_html.html

<img src="https://i.ibb.co/9tQPW60/100-days.png">
<img src="https://thumbs.gfycat.com/LiquidQuestionableGrassspider-max-1mb.gif"></center>
<br> <center> On January 25th, President Joe Biden pledged 150 million vaccinations in 100 days. </center>

In [166]:
us_vaccinations_df['daily_vaccinations'] = us_vaccinations_df['total_vaccinations'] - us_vaccinations_df['total_vaccinations'].shift(1)

In [167]:
'''Fit the model by instantiating a new Prophet object. Using logistics settings to the forecasting procedure.
Call its fit method and pass in the historical dataframe.'''

total_vacc = us_vaccinations_df.rename(columns={'date': 'ds', 'total_vaccinations': 'y'})
total_vacc['cap'] = 328000000
m = Prophet(growth='logistic')
m.fit(total_vacc)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [168]:
'''ds column contains dates for which a prediction is to be made. You can get a suitable dataframe that 
extends into the future a specified number of days using the helper method Prophet.make_future_dataframe.'''

future_total_vacc = m.make_future_dataframe(periods=71)
future_total_vacc['cap'] = 328000000
print(future_total_vacc.tail())

            ds        cap
202 2021-07-24  328000000
203 2021-07-25  328000000
204 2021-07-26  328000000
205 2021-07-27  328000000
206 2021-07-28  328000000


In [169]:
'''The predict method will assign each row in future a predicted value which it names yhat. The forecast object 
here is a new dataframe that includes a column yhat with the forecast, as well as columns for components and 
uncertainty intervals.'''

forecast_total_vacc = m.predict(future_total_vacc)
print(forecast_total_vacc[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

            ds         yhat   yhat_lower   yhat_upper
202 2021-07-24 323493647.24 321400270.35 325128092.96
203 2021-07-25 324013837.04 321990910.55 325682192.25
204 2021-07-26 323367952.53 321295441.53 325033324.11
205 2021-07-27 323123400.36 321008755.70 324704434.95
206 2021-07-28 322900795.04 320764687.64 324465559.23


In [170]:
total_vacc_pred_fig = plot_plotly(m, forecast_total_vacc)
total_vacc_pred_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
total_vacc_pred_fig.update_layout(title_text='Prediction of Total US Vaccinations', title_x=0.5)
total_vacc_pred_fig.show()

In [171]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

total_vacc_pred_fig.write_html('/home/mayoubi/COVID_data/Images/total_vacc_pred_fig_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/total_vacc_pred_fig_html.html

<img src="https://i.ibb.co/x3b4wcK/reach-out.png">
<center><a href="http://bit.ly/vizvax2021">bit.ly/vizvax2021</a></center>
<img src="https://i.ibb.co/mBNMpHY/contact-info.png">